In [1]:
import numpy as np
import pandas as pd

from sklearn import datasets

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
# Read in data
mod_data = pd.read_csv('appointments_clean.csv')

In [3]:
# Split into X and y
X = mod_data.drop(
    ['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay', 
     'Neighbourhood', 'No-show', 'NoShow'], 
    axis = 1
)
y = mod_data.NoShow

# Test/train split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 314
)

In [4]:
# Scale data
scaler = MinMaxScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
# KNN
clf_kn  = KNeighborsClassifier().fit(X_train, y_train)
pred_kn = clf_kn.predict(X_test)

In [6]:
# Logistic Regression (L2)
clf_lr  = LogisticRegression(penalty = 'l2').fit(X_train, y_train)
pred_lr = clf_lr.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [7]:
# Logistic Regression (L1)
clf_rr  = LogisticRegression(penalty = 'l1').fit(X_train, y_train)
pred_rr = clf_rr.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
# Gaussian Process
clf_gp  = GaussianProcessClassifier().fit(X_train, y_train)
pred_gp = clf_gp.predict(X_test)

In [ ]:
# Neural Net
clf_nn  = MLPClassifier().fit(X_train, y_train)
pred_nn = clf_nn.predict(X_test)

In [ ]:
# Check model performance - binary outcome
print('KNN')
print(classification_report(y_test, pred_kn))

print('L2 Regularized Logistic Regression')
print(classification_report(y_test, pred_lr))

print('L1 Regularized Logistic Regression')
print(classification_report(y_test, pred_rr))

print('Gaussian Process')
print(classification_report(y_test, pred_gp))

print('Neural Network')
print(classification_report(y_test, pred_nn))

In [ ]:
# Probability Assessment Function
def prob_summary(clf, data, actual):
    
    # Generate predicted probabilities
    probs = clf.predict_proba(data)
    probs = pd.Series([i[1] for i in probs])
    
    # Add probabilities to actuals in dataframe
    probs_df = pd.concat([probs, actual.reset_index()], axis = 1).reset_index()
    
    # Clean dataframe
    probs_df.columns = ['level_0', 'Prob', 'index', 'Actual']
    probs_df         = probs_df.drop(['level_0', 'index'], axis = 1)
    
    # Round probabilities to nearest tenth
    probs_df['RoundedProb'] = (probs_df['Prob']).round(1)
    
    # Summarize
    summary = probs_df.groupby(
        'RoundedProb'
    ).agg(
        {
            'Actual':['count', 'sum']
        }
    )
    
    # Fix column names
    summary.columns = ["_".join(i) for i in summary.columns.ravel()]
    
    # Calculate predicted percentages
    summary['Actual_percentage'] = (summary['Actual_sum'] / summary['Actual_count']).round(2)
    return(summary)

In [ ]:
# Probability Assessment
# Check model performance - probabilities
print('KNN')
print(prob_summary(clf_kn, X_test, y_test), '\n')

print('L2 Regularized Logistic Regression')
print(prob_summary(clf_lr, X_test, y_test), '\n')

print('L1 Regularized Logistic Regression')
print(prob_summary(clf_rr, X_test, y_test), '\n')

print('Gaussian Process')
print(prob_summary(clf_qp, X_test, y_test), '\n')

print('Neural Network')
print(prob_summary(clf_nn, X_test, y_test), '\n')